In [2]:
%matplotlib inline

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import os, sys, shutil
sys.path.append("..")
import pandas as pd

from lammpsrun import LAMMPS, Prism
from PBSJobSubmissionScript import PBS_Submitter, qstat_monitor
from utility_scripts import *

from ase import Atoms, units
from ase.visualize import view
from ase.visualize.plot import plot_atoms
from ase.io.trajectory import Trajectory
from ase.io import write, read
from ase.build import molecule, bcc110, bcc111
from ase.spacegroup import crystal
from ase.data import atomic_numbers, atomic_names, atomic_masses, covalent_radii

cx1_out = r"/rds/general/user/yx6015/home/cx1_out/"
ephemeral = r"/rds/general/user/yx6015/ephemeral/"
sim_path = r"/rds/general/user/yx6015/home/LAMMPS_Simulation/HPC_Jupyter/"
print(cx1_out + '\n' + sim_path)
potential = "fe_o_water_fe_o_c_full_water.reax"

/rds/general/user/yx6015/home/cx1_out/
/rds/general/user/yx6015/home/LAMMPS_Simulation/HPC_Jupyter/


# MKL/Multiprocessing Test

In [2]:
file_name = 'test'
lammps_traj_file = 'test.lammpstrj'
mol_file = file_name + ".extxyz"
ase_traj_file = file_name + ".traj"

In [ ]:
%%prun -l 10
import sys
import time
sys.path.append("..")
from ase import Atoms, units
from ase.io.trajectory import Trajectory
from ase.io import write, read
from HPC_Scripts.minimal_traj_conversion import LAMMPS, Prism
import mkl
mkl.set_num_threads(8)

_start = time.time()

calc = LAMMPS()
mol = read(mol_file)
mol.set_calculator(calc)
calc.atoms = mol
calc.prism = Prism(mol.get_cell())

print("Writing ASE trajectory to ", ase_traj_file)
calc.trajectory_out = Trajectory(ase_traj_file, 'w')

print("Reading LAMMPS Trajectory from", lammps_traj_file)
calc.read_lammps_trj(lammps_trj = lammps_traj_file, n_cpus=8)
calc.trajectory_out.close()

print("Took {} seconds".format(time.time() - _start))

Writing ASE trajectory to  test.traj
Reading LAMMPS Trajectory from test.lammpstrj


Process Process-7:
Traceback (most recent call last):
  File "/rds/general/user/yx6015/home/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/rds/general/user/yx6015/home/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "../HPC_Scripts/minimal_traj_conversion.py", line 144, in step_writer
    cell=cell_atoms)
  File "/rds/general/user/yx6015/home/anaconda3/lib/python3.7/site-packages/ase/atoms.py", line 223, in __init__
    self.new_array('positions', positions, float, (3,))
  File "/rds/general/user/yx6015/home/anaconda3/lib/python3.7/site-packages/ase/atoms.py", line 416, in new_array
    (len(a), len(b)))
ValueError: Array has wrong length: 3600 != 10688.


In [ ]:
%%prun -l 10
import sys
import time
sys.path.append("..")
from ase import Atoms, units
from ase.io.trajectory import Trajectory
from ase.io import write, read
from minimal_traj_conversion_queue import LAMMPS, Prism
import mkl
mkl.set_num_threads(8)

_start = time.time()

calc = LAMMPS()
mol = read(mol_file)
mol.set_calculator(calc)
calc.atoms = mol
calc.prism = Prism(mol.get_cell())

print("Writing ASE trajectory to ", ase_traj_file)
calc.trajectory_out = Trajectory(ase_traj_file, 'w')

print("Reading LAMMPS Trajectory from", lammps_traj_file)
calc.read_lammps_trj(lammps_trj = lammps_traj_file)
calc.trajectory_out.close()

print("Took {} seconds".format(time.time() - _start))

In [2]:
%%prun -l 10
import sys
import time
sys.path.append("..")
from ase import Atoms, units
from ase.io.trajectory import Trajectory
from ase.io import write, read
from lammpsrun import LAMMPS, Prism
import mkl
mkl.set_num_threads(8)

_start = time.time()

calc = LAMMPS()
mol = read(mol_file)
mol.set_calculator(calc)
calc.atoms = mol
calc.prism = Prism(mol.get_cell())

print("Writing ASE trajectory to ", ase_traj_file+"1")
calc.trajectory_out = Trajectory(ase_traj_file+"1", 'w')

print("Reading LAMMPS Trajectory from", lammps_traj_file)
calc.read_lammps_trj(lammps_trj = lammps_traj_file)
calc.trajectory_out.close()

print("Took {} seconds".format(time.time() - _start))

Writing ASE trajectory to  test.traj1
Reading LAMMPS Trajectory from test.lammpstrj
Took 18.122143268585205 seconds
 

         11819424 function calls (11758968 primitive calls) in 18.265 seconds

   Ordered by: internal time
   List reduced from 2828 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      650    4.744    0.007    5.038    0.008 {method 'read' of 'pandas._libs.parsers.TextReader' objects}
     1303    2.100    0.002    2.100    0.002 {method 'tofile' of 'numpy.ndarray' objects}
      650    1.155    0.002    1.178    0.002 parsers.py:1830(__init__)
      651    0.777    0.001    1.209    0.002 symbols.py:61(symbols2numbers)
     3802    0.743    0.000    0.743    0.000 {built-in method posix.stat}
2346896/2343295    0.687    0.000    0.794    0.000 {built-in method builtins.next}
  3114720    0.587    0.000    0.816    0.000 {built-in method builtins.isinstance}
        1    0.541    0.541   17.982   17.982 lammpsrun.py:569(read_lammps_trj)
38497/37847    0.354    0.000    0.363    0.000 {built-in method numpy.array}
      650    0.

# Conversion Speed Test

In [2]:
aq_out = '2466596.cx1'
aq_name = 'Aq_efield_0MVcm'
out_dir = os.path.join(ephemeral, aq_out)
convert_scripts_dir = r"/rds/general/user/yx6015/home/LAMMPS_Simulation/HPC_Scripts/"

In [12]:
test_list = ['', '_nocopy', '_nocopy_nosend', '_fast']
script_list = ["convert_script{}.py".format(x) for x in test_list]
test_args = ['{}'.format(aq_name),
             '{} {}'.format(aq_name, os.path.join(out_dir, aq_name+'.lammpstrj')),
             '{} {} {}'.format(aq_name, os.path.join(out_dir, aq_name+'.lammpstrj'), os.path.join(out_dir, aq_name+'.traj1')),
             '{}'.format(aq_name)]
test_commands = [["python {} {}".format(x[0], x[1])] for x in zip(script_list, test_args)]

In [13]:
test_source_files = [[os.path.join(out_dir, "*extxyz"), 
                      os.path.join(convert_scripts_dir, x),
                      os.path.join(convert_scripts_dir, "lammpsrun.py")] for x in script_list]
test_source_files[0] += [os.path.join(out_dir, "*lammpstrj")]
test_source_files[3] += [os.path.join(out_dir, "*lammpstrj")]
test_names = ['convert{}'.format(x) for x in test_list]

In [14]:
convert_PBS = PBS_Submitter(job_names=test_names,
                            job_commands=test_commands,
                            modules=["mpi", "anaconda3/personal"],
                            walltime="24:00:00",
                            proc_nodes=1,
                            proc_cpus=8,
                            proc_mpiprocs=8, 
                            memory=32,
                            source_files=test_source_files)

In [15]:
convert_out, convert_err = convert_PBS.run()
print(convert_out)
print(test_names)

['2475004.cx1', '2475005.cx1', '2475006.cx1', '2475007.cx1', '2475008.cx1']
['convert', 'convert_nocopy', 'convert_nocopy_nosend', 'convert_fast', 'convert_nocopy_nosend_fast']


### Final results:
- *fast* is slightly faster (by ~9%)
- *nocopy* is faster than both *nosend* (by 30%) and *normal* (by ~44%)

### Conclusion
Use mkl acceleration and do not copy over lammpstrj

# Archiving FYP Runs

### List of Files

In [3]:
job_outs = []

In [4]:
# Pure Water Polarisation under electric field
job_outs.extend(['123895.pbs', '123896.pbs', '123897.pbs', '123898.pbs', '123899.pbs', '123900.pbs', '123901.pbs', '123902.pbs', '123903.pbs', '123904.pbs', '123905.pbs', '123906.pbs', '123907.pbs', '123908.pbs', '123909.pbs', '123910.pbs', '123911.pbs', '123912.pbs', '123913.pbs', '123914.pbs', '123915.pbs'])
# Surface Energy
job_outs.extend(['127330.pbs', '127331.pbs', '127332.pbs', '127333.pbs', '127334.pbs', '127335.pbs', '127336.pbs', '127337.pbs', '127338.pbs', '127339.pbs', '127340.pbs', '127341.pbs', '127342.pbs', '127343.pbs', '127344.pbs', '127345.pbs', '127346.pbs', '127347.pbs', '127348.pbs', '127349.pbs', '127350.pbs', '127351.pbs', '127352.pbs', '127353.pbs', '127354.pbs', '127355.pbs', '127356.pbs', '127357.pbs', '127358.pbs', '127359.pbs', '127360.pbs', '127361.pbs', '127369.pbs', '127370.pbs', '127372.pbs', '127373.pbs', '127374.pbs', '127375.pbs', '127420.pbs', '127421.pbs', '127422.pbs', '127423.pbs', '127424.pbs', '127425.pbs', '127426.pbs', '127429.pbs', '127432.pbs', '127433.pbs', '127434.pbs', '127435.pbs', '127436.pbs', '127443.pbs', '127444.pbs', '127445.pbs', '127446.pbs', '127447.pbs', '127450.pbs', '127452.pbs', '127453.pbs', '127454.pbs', '127455.pbs', '127458.pbs', '127459.pbs', '127465.pbs', '127466.pbs', '127467.pbs', '127468.pbs', '127471.pbs', '127481.pbs', '127482.pbs', '127483.pbs', '127484.pbs', '127485.pbs', '127486.pbs', '127487.pbs', '127494.pbs', '127495.pbs', '127496.pbs', '127497.pbs', '127498.pbs', '127499.pbs', '127500.pbs', '127501.pbs', '127502.pbs', '127503.pbs', '127504.pbs', '127505.pbs', '127506.pbs', '127507.pbs', '127510.pbs', '127511.pbs', '127512.pbs', '127513.pbs', '127514.pbs', '127517.pbs', '127518.pbs', '127522.pbs', '127523.pbs', '127525.pbs', '127526.pbs', '127527.pbs', '127528.pbs', '127529.pbs', '127531.pbs', '127532.pbs', '127533.pbs', '127534.pbs', '127535.pbs', '127536.pbs', '127537.pbs', '127538.pbs', '127539.pbs', '127540.pbs', '127541.pbs', '127561.pbs', '127562.pbs', '127563.pbs', '127564.pbs', '127565.pbs', '127566.pbs', '127568.pbs', '127569.pbs', '127570.pbs', '127571.pbs'])
# Fe Aqueous + CO2 Large
job_outs.extend(['127575.pbs', '127576.pbs', '127577.pbs', '127579.pbs'])
# Fe Aqueous + CO2 Small Long
job_outs.extend(['126387.pbs', '126388.pbs', '126389.pbs', '126390.pbs'])
# Fe Aqueous + CO2 Small
job_outs.extend(['126448.pbs', '126449.pbs', '126450.pbs', '126451.pbs'])
# Fe Aqueous Small
job_outs.extend(['123766.pbs', '123767.pbs', '123768.pbs', '123769.pbs'])

In [15]:
total_size = 0
total_files = 0
for job in job_outs:
    job_path = os.path.join(ephemeral, job)
    all_files = os.listdir(job_path)
    total_size += sum([os.path.getsize(os.path.join(job_path, x)) for x in all_files])
    total_files += len(all_files)
print("There are a total of {} files".format(total_files))
print("The files occupy a total of {:.2f} GB".format(total_size/1024**3))

There are a total of 1480 files
The files occupy a total of 387.24 GB


In [17]:
for job in job_outs:
    job_path = os.path.join(ephemeral, job)
    fake_ephemeral = r"/rds/general/user/yx6015/home/fake_ephemeral/"
    dest_path  = os.path.join(fake_ephemeral, job)
    shutil.copytree(job_path, dest_path)